In [48]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [49]:
import torch_geometric
import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv

import utils.torch_utils
from model.ltl.batched_transition_graph import BatchedTransitionGraph
from src.ltl.automata import ltl2ldba
from src.ltl.logic import Assignment
from src.model.ltl import TransitionGraph
from src.utils import torch_utils

In [50]:
formula = 'F green'
ldba = ltl2ldba(formula, propositions=frozenset({'green', 'red', 'yellow', 'blue'}), simplify_labels=False)
assert ldba.check_valid()
ldba.complete_sink_state()
ldba.prune_impossible_transitions(Assignment.more_than_one_true_proposition(ldba.propositions))
tg_green = TransitionGraph.from_ldba(ldba)
tg_green.validate()

True

In [51]:
formula = 'F yellow'
ldba = ltl2ldba(formula, propositions=frozenset({'green', 'red', 'yellow', 'blue'}), simplify_labels=False)
assert ldba.check_valid()
ldba.complete_sink_state()
ldba.prune_impossible_transitions(Assignment.more_than_one_true_proposition(ldba.propositions))
tg_yellow = TransitionGraph.from_ldba(ldba)
tg_yellow.validate()

True

In [52]:
batch = BatchedTransitionGraph([tg_green, tg_yellow], active_transitions=[[0], [0]])

In [53]:
tg = batch.all()
tg.x

tensor([[1., 1., 1., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0., 0., 1.],
        [1., 1., 1., 1., 1., 0., 1.],
        [1., 0., 1., 1., 1., 0., 0.],
        [0., 1., 0., 0., 0., 0., 1.],
        [1., 1., 1., 1., 1., 0., 1.]])

In [54]:
tg.edge_index

tensor([[0, 1, 2, 2, 3, 4, 5, 5],
        [0, 0, 2, 1, 3, 3, 5, 4]])

In [56]:
from ltl import EventuallySampler
from envs import make_env
from model import build_model
from config import model_configs
env_name = 'ltl_point_mass'
exp = 'try'
seed = 1

env = make_env(env_name, EventuallySampler)
config = model_configs['default']
status = torch.load(f'experiments/ppo/{env_name}/{exp}/{seed}/status.pth', map_location='cpu')
model = build_model(env, {}, config)

Num GNN parameters: 128


In [57]:
model.ltl_net(batch)

tensor([[ 0.3151, -0.4744, -0.3429, -0.0042, -0.0442, -0.1536,  0.6108,  0.1626,
         -0.2558, -0.6381, -0.2399, -0.0169,  0.2209,  0.1656, -0.0635,  0.3227],
        [ 0.3151, -0.4744, -0.3429, -0.0042, -0.0442, -0.1536,  0.6108,  0.1626,
         -0.2558, -0.6381, -0.2399, -0.0169,  0.2209,  0.1656, -0.0635,  0.3227]],
       grad_fn=<ScatterAddBackward0>)

In [23]:
# TODO: GNN pretraining to learn distinct embeddings hopefully. Can do this in the MDP where actions correspond to choosing labels. Do I then even need to update the GNN during training? Read through LTL2Action paper again.
# TODO: is there any way I can prevent sums of embeddings from being the same?
# TODO: why is SPS lower even with embedding implementation? Is it the LDBA parsing or something else?